In [ ]:
# %load yelpSample.py

"""
Yelp API v2.0 code sample.

This program demonstrates the capability of the Yelp API version 2.0
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.

Please refer to http://www.yelp.com/developers/documentation for the API documentation.

This program requires the Python oauth2 library, which you can install via:
`pip install -r requirements.txt`.

Sample usage of the program:
`python yelpSample.py --term="bars" --location="San Francisco, CA"`
"""
import argparse
import json
import pprint
import sys
import urllib
import urllib2

import oauth2


API_HOST = 'api.yelp.com'
DEFAULT_TERM = 'dinner'
DEFAULT_LOCATION = 'San Francisco, CA'
SEARCH_LIMIT = 3
SEARCH_PATH = '/v2/search/'
BUSINESS_PATH = '/v2/business/'

# OAuth credential placeholders that must be filled in by users.
CONSUMER_KEY = None
CONSUMER_SECRET = None
TOKEN = None
TOKEN_SECRET = None

import sys
sys.path.append('/Users/yoavfreund/VaultDSE/')
import yelpCreds
CONSUMER_KEY,CONSUMER_SECRET,TOKEN,TOKEN_SECRET=yelpCreds.getKeys()

def request(host, path, url_params=None):
    """Prepares OAuth authentication and sends the request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        urllib2.HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = 'http://{0}{1}?'.format(host, path)

    consumer = oauth2.Consumer(CONSUMER_KEY, CONSUMER_SECRET)
    oauth_request = oauth2.Request(method="GET", url=url, parameters=url_params)

    oauth_request.update(
        {
            'oauth_nonce': oauth2.generate_nonce(),
            'oauth_timestamp': oauth2.generate_timestamp(),
            'oauth_token': TOKEN,
            'oauth_consumer_key': CONSUMER_KEY
        }
    )
    token = oauth2.Token(TOKEN, TOKEN_SECRET)
    oauth_request.sign_request(oauth2.SignatureMethod_HMAC_SHA1(), consumer, token)
    signed_url = oauth_request.to_url()
    
    print 'Querying {0} ...'.format(url)

    conn = urllib2.urlopen(signed_url, None)
    try:
        response = json.loads(conn.read())
    finally:
        conn.close()

    return response

def search(term, location):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """
    
    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, url_params=url_params)

def get_business(business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path)

def query_api(term, location):
    """Queries the API by the input values from the user.

    Args:
        term (str): The search term to query.
        location (str): The location of the business to query.
    """
    response = search(term, location)

    businesses = response.get('businesses')

    if not businesses:
        print 'No businesses for {0} in {1} found.'.format(term, location)
        return

    business_id = businesses[0]['id']

    print '{0} businesses found, querying business info for the top result "{1}" ...'.format(
        len(businesses),
        business_id
    )

    response = get_business(business_id)

    print 'Result for business "{0}" found:'.format(business_id)
    pprint.pprint(response, indent=2)


def main():
    parser = argparse.ArgumentParser()

    parser.add_argument('-q', '--term', dest='term', default=DEFAULT_TERM, type=str, help='Search term (default: %(default)s)')
    parser.add_argument('-l', '--location', dest='location', default=DEFAULT_LOCATION, type=str, help='Search location (default: %(default)s)')

    input_values = parser.parse_args()

    try:
        query_api(input_values.term, input_values.location)
    except urllib2.HTTPError as error:
        sys.exit('Encountered HTTP error {0}. Abort program.'.format(error.code))


if __name__ == '__main__':
    main()


In [1]:
# get some help
!python yelpSample.py -h

usage: yelpSample.py [-h] [-q TERM] [-l LOCATION]

optional arguments:
  -h, --help            show this help message and exit
  -q TERM, --term TERM  Search term (default: dinner)
  -l LOCATION, --location LOCATION
                        Search location (default: San Francisco, CA)


In [2]:
!python yelpSample.py --term="sushi ota" --location="La Jolla, CA"

Querying http://api.yelp.com/v2/search/? ...
3 businesses found, querying business info for the top result "sushi-ota-san-diego" ...
Querying http://api.yelp.com/v2/business/sushi-ota-san-diego? ...
Result for business "sushi-ota-san-diego" found:
{ u'categories': [[u'Sushi Bars', u'sushi'], [u'Japanese', u'japanese']],
  u'display_phone': u'+1-858-270-5670',
  u'id': u'sushi-ota-san-diego',
  u'image_url': u'https://s3-media3.fl.yelpcdn.com/bphoto/bySWiib8tOs8lPO-s7X7Sw/ms.jpg',
  u'is_claimed': False,
  u'is_closed': False,
  u'location': { u'address': [u'4529 Mission Bay Dr'],
                 u'city': u'San Diego',
                 u'coordinate': { u'latitude': 32.8036503,
                                  u'longitude': -117.2171111},
                 u'country_code': u'US',
                 u'cross_streets': u'Bunker Hill St',
                 u'display_address': [ u'4529 Mission Bay Dr',
                                       u'Pacific Beach',
                                    

#### excercise

Write a script that returns a web page that contains all of the information about a single restaurant, including the larger images (not starts, for example).